In [99]:
import numpy as np
import pandas as pd
import plotly.express as px
import dataframe_image as dfi

In [2]:
#define color palettes to use
continous_color_pallete_1 = px.colors.sequential.Viridis
continous_color_pallete_2 = px.colors.sequential.Blues
discrete_color_pallete = px.colors.qualitative.G10

In [3]:
returns_columns = ["1MReturn","3MReturn","6MReturn","12MReturn"]

Importing data

In [4]:
convictions_and_returns = pd.read_csv('convictions_and_returns.csv')  

In [5]:
## formating date
convictions_and_returns['DATE'] = pd.to_datetime(
                          convictions_and_returns['DATE'],
                          format='%Y-%m-%d')

In [6]:
## sorting
convictions_and_returns = convictions_and_returns.sort_values(by="DATE")

In [7]:
# replace -1 with nan in Returns columns
convictions_and_returns[returns_columns] = convictions_and_returns[returns_columns].replace({'-1':np.nan, -1:np.nan})

In [8]:
# drop duplicated index

convictions_and_returns.drop(['Unnamed: 0'],axis=1, inplace=True)

are some rows duplicated?

In [9]:
convictions_and_returns.columns

Index(['DATE', 'ID', 'STOCK', 'SECTOR', 'TYPE', 'SCORE', 'TIME', '1MReturn',
       '3MReturn', '6MReturn', '12MReturn'],
      dtype='object')

In [10]:
convictions_and_returns.duplicated().sum()

0

How many stocks dont have any ratings

In [11]:
is_all_null_per_stock = convictions_and_returns.groupby('STOCK')['SCORE'].apply(lambda x: x.isnull().all()).to_frame()
all_null_per_stock_count = is_all_null_per_stock['SCORE'].value_counts().to_frame()
all_null_per_stock_count.reset_index(inplace=True)
all_null_per_stock_count.columns = ['VALUE', 'COUNT']

In [14]:
all_null_per_stock_count

,VALUE,COUNT
0,False,608
1,True,13


In [19]:
fig = px.bar(all_null_per_stock_count, x='VALUE', y='COUNT', color = 'VALUE', title = 'Does the Company have non ratings?',
            color_discrete_sequence=discrete_color_pallete,
            labels={
                     "COUNT": "Number of Companies",
                     "VALUE": "Have non ratings?"
                 },)
fig.show()

Companies whitout any ratings can be deleted. There is nothing more to analyze

In [20]:
# deleting companies without any ratings
companies_without_scores = is_all_null_per_stock.loc[is_all_null_per_stock['SCORE'] == True]
companies_without_scores.reset_index(inplace=True)
companies_without_score = companies_without_scores['STOCK'].to_list()

In [21]:
convictions_and_returns = convictions_and_returns[~convictions_and_returns['STOCK'].isin(companies_without_score)]

Duplicated values in-depth analysis

In [22]:
convictions_and_returns.duplicated(subset=["STOCK","DATE"]).sum()

115

In [23]:
convictions_and_returns.duplicated(subset=["STOCK","DATE","SCORE"]).sum()

115

The duplicated rows can be removed, because more or two scores for the same date and stock dont bring any new information

In [24]:
convictions_and_returns.drop_duplicates(subset=["STOCK","DATE","SCORE"],keep="first",inplace=True)

In [25]:
## save deduplicated convictions and returns without companies with no convictions

convictions_and_returns.to_csv(r'Assets\convictions_and_returns_cleaned.csv')

How many Stocks have at least one missing score?

In [26]:
is_any_null_per_stock = convictions_and_returns.groupby('STOCK')['SCORE'].apply(lambda x: x.isnull().any()).to_frame()

In [27]:
any_null_per_stock_count = is_any_null_per_stock['SCORE'].value_counts().to_frame()
any_null_per_stock_count.reset_index(inplace=True)
any_null_per_stock_count.columns = ['VALUE', 'COUNT']

In [29]:
fig = px.pie(any_null_per_stock_count, values='COUNT', names='VALUE', title='Does Company have at least one missing rating?',
            color_discrete_sequence=discrete_color_pallete)
fig.show()

Almost 80 percent of companies have at least one measing value when it comes to score

Which Percent of Returns are missing

In [30]:
percent_missing = convictions_and_returns[returns_columns].isnull().sum() * 100 / len(convictions_and_returns)

In [31]:
percent_missing

1MReturn     0.164745
3MReturn     0.114605
6MReturn     0.100279
12MReturn    4.440943
dtype: float64

How many companies dont have any returns?

In [32]:
is_all_returns_null_per_stock = convictions_and_returns.groupby('STOCK')[returns_columns].apply(lambda x: x.isnull().all())
is_all_returns_per_stock_count = is_all_returns_null_per_stock[returns_columns].value_counts()

In [33]:
is_all_returns_per_stock_count

1MReturn  3MReturn  6MReturn  12MReturn
False     False     False     False        608
dtype: int64

 there is no companies left witch dont have any 

Consequtives nans for each company

In [34]:
list_of_companies = convictions_and_returns["STOCK"].unique()
list_of_dates = convictions_and_returns["DATE"].unique()

In [35]:
def extract_nan_statistics(df, list_of_companies):
    nulls = pd.DataFrame(columns=["STOCK","NAN number",'Consequtive NAN max' ])
    for l in list_of_companies:
        stock_data = df.loc[df['STOCK'] == l]
        temp = stock_data.SCORE.isnull().astype(int).groupby(stock_data.SCORE.notnull().astype(int).cumsum()).sum() ## consequtives nans
        nulls.loc[len(nulls.index)] = [l, temp.sum(), temp.max()]
        
    return nulls


In [36]:
nulls = extract_nan_statistics(convictions_and_returns, list_of_companies)

In [37]:
nulls_max_value_counts = nulls['Consequtive NAN max'].value_counts().to_frame()
nulls_max_value_counts.reset_index(inplace = True)

In [37]:
nulls_max_value_counts.columns

Index(['index', 'Consequtive NAN max'], dtype='object')

In [42]:
fig = px.bar(nulls_max_value_counts, x='index', y='Consequtive NAN max', color= 'Consequtive NAN max',
              labels={
                     "index": "Number of consecutive nulls in score",
                     "Consequtive NAN max": "Number of companies"
                 },
                 title = "Numbers of consecutive nulls in score by Companies",
                 color_continuous_scale=continous_color_pallete_1,
                 text_auto=True 
                )
fig.show()

In [43]:
print(len(list_of_dates))

23


How many companies have more or equal than 17 consequtive nans?

In [43]:
companies_with_17NAN = nulls_max_value_counts.query("index >= 17")['Consequtive NAN max'].sum() 

companies_with_17NAN/len(list_of_companies)

0.1513157894736842

But how many of which is just 20 consequtive nans?

In [44]:
companies_with_20NAN = nulls_max_value_counts.query("index == 20")['Consequtive NAN max'].sum() 

companies_with_20NAN/len(list_of_companies)

0.125

Most of them come from 20 consequtive nans which is definately two much, so we decided to excluded stocks with more than 17 consequtive nans

In [45]:
nulls.rename(columns = {'Consequtive NAN max':'Consequtive_NAN_Max'}, inplace = True)
companies_with_17NAN = nulls.query('Consequtive_NAN_Max >= 17')['STOCK'].tolist()

In [46]:
convictions_and_returns = convictions_and_returns[~convictions_and_returns['STOCK'].isin(companies_with_17NAN)]

Calculating new NAN statistics

In [47]:
list_of_companies = convictions_and_returns["STOCK"].unique()

In [64]:
nulls = extract_nan_statistics(convictions_and_returns, list_of_companies)
nulls_max_value_counts = nulls['Consequtive NAN max'].value_counts().to_frame()
nulls_max_value_counts.reset_index(inplace = True)

Which part of nans is only one period specific?

In [65]:
nulls['one_period_only'] = np.where(nulls['NAN number']== nulls["Consequtive NAN max"], True, False)

In [67]:
nulls.columns = ['STOCK',"NAN_number","Consequtive_NAN_max","one_period_only"]

In [68]:
is_nan_one_period_only = nulls[nulls["Consequtive_NAN_max"] != 0]
is_nan_one_period_only_count = is_nan_one_period_only['one_period_only'].value_counts().to_frame()
is_nan_one_period_only_count.reset_index(inplace=True)
is_nan_one_period_only_count.columns = ['VALUE', 'COUNT']

In [69]:
is_nan_one_period_only_count

,VALUE,COUNT
0,True,302
1,False,76


In [71]:
fig = px.pie(is_nan_one_period_only_count, values='COUNT', names='VALUE', title='Does all nulls appear in one period of time by Company?',
            color_discrete_sequence=discrete_color_pallete)
fig.show()

In [55]:

nulls_not_only_consequtive = nulls.query('one_period_only == False')

In [56]:
nulls_not_only_consequtive_max_value_counts = nulls_not_only_consequtive['Consequtive NAN max'].value_counts().to_frame()
nulls_not_only_consequtive_max_value_counts.reset_index(inplace = True)

In [57]:
fig = px.bar(nulls_not_only_consequtive_max_value_counts, x='index', y='Consequtive NAN max', color= 'Consequtive NAN max',
              labels={
                     "index": "number of consequtive nulls",
                     "Consequtive NAN max": "Number of companies"
                 },
                 title = "Numbers of consequtive nulls in Companies",
                 color_continuous_scale=continous_color_pallete_1 
                )
fig.show()

WE decided to delete consequtive nans but save non consequtive

In [72]:
def drop_3_consequtive_nans(df, list_of_companies):
    df_to_return = pd.DataFrame()
    for l in list_of_companies:
        stock_data = df.loc[df['STOCK'] == l]
        stock_data['SCORE_LAG'] = stock_data["SCORE"].shift(-1)
        stock_data['SCORE_LEAD'] = stock_data['SCORE'].shift(1)
        stock_data.dropna(subset=['SCORE','SCORE_LAG','SCORE_LEAD'], how='all', inplace = True)
  
        if df_to_return.empty == True:
            df_to_return = stock_data
            print('first_time')
        else:
            df_to_return = pd.concat([df_to_return, stock_data], ignore_index=True)

        
    return df_to_return

In [73]:
len(convictions_and_returns)

11848

In [ ]:
convictions_test = drop_3_consequtive_nans(convictions_and_returns, list_of_companies)


In [208]:
convictions_test

,DATE,ID,STOCK,SECTOR,TYPE,SCORE,TIME,1MReturn,3MReturn,6MReturn,12MReturn,SCORE_LAG,SCORE_LEAD
0,2004-02-11,GN63J3-R,SU,Energy Minerals,MERGED,0.953727,10:01:54.391,0.017659,-0.083391,0.052675,0.416997,0.953686,NaN
1,2004-02-18,GN63J3-R,SU,Energy Minerals,MERGED,0.953686,10:01:55.714,0.071475,-0.078276,0.077998,0.479119,0.940502,0.953727
2,2004-02-25,GN63J3-R,SU,Energy Minerals,MERGED,0.940502,10:01:56.304,0.038620,-0.044470,0.089185,0.572897,0.943390,0.953686
3,2004-03-03,GN63J3-R,SU,Energy Minerals,MERGED,0.943390,10:01:57.217,-0.014190,-0.107854,0.085261,0.478450,0.942921,0.940502
4,2004-03-10,GN63J3-R,SU,Energy Minerals,MERGED,0.942921,10:01:58.759,0.002599,-0.093852,0.062814,0.433562,0.935887,0.943390
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8561,2004-06-02,LTP0Z2-R,CSX,Transportation,MERGED,0.180638,10:02:09.320,-0.010585,0.002225,0.194983,0.321101,0.181605,0.187122
8562,2004-06-09,LTP0Z2-R,CSX,Transportation,MERGED,0.181605,10:02:10.134,-0.033579,0.011197,0.175649,0.292626,0.181764,0.180638
8563,2004-06-16,LTP0Z2-R,CSX,Transportation,MERGED,0.181764,10:02:10.676,-0.032350,0.055776,0.232342,0.382558,0.183313,0.181605
8564,2004-06-23,LTP0Z2-R,CSX,Transportation,MERGED,0.183313,10:02:11.490,-0.077629,0.034432,0.225329,0.318808,NaN,0.181764


In [75]:

len(convictions_test)

8566

In [76]:
list_of_companies = convictions_test["STOCK"].unique()

In [77]:
nulls = extract_nan_statistics(convictions_test, list_of_companies)
nulls_max_value_counts = nulls['Consequtive NAN max'].value_counts().to_frame()
nulls_max_value_counts.reset_index(inplace = True)


In [80]:
fig = px.bar(nulls_max_value_counts, x='index', y='Consequtive NAN max', color= 'Consequtive NAN max',
              labels={
                     "index": "Number of consecutive nulls in score",
                     "Consequtive NAN max": "Number of companies"
                 },
                 title = "Numbers of consecutive nulls in score by Companies",
                 color_continuous_scale=continous_color_pallete_2  
                )
fig.show()

In [213]:
convictions_test = convictions_test.assign(
                   SCORE =convictions_test.SCORE.fillna(method='ffill'),
                   )

DROP not needed columns

In [214]:
convictions_test.drop(['SCORE_LAG', 'SCORE_LEAD'], axis=1, inplace = True)

Now there are no missing values in convictions, but might be in prices

first if there is a observations without any returns than it can be dropped

In [215]:
convictions_test.dropna(subset=['1MReturn','3MReturn','6MReturn','12MReturn'], how='all', inplace = True)

In [93]:
null_numbers = convictions_test.isnull().sum(axis = 0)
null_numbers = null_numbers.to_frame()
null_numbers.reset_index(inplace=True)

In [97]:
null_numbers.columns = ['Column','Nan_number']
null_numbers = null_numbers[null_numbers['Column'].isin(returns_columns)] 

In [100]:
nulls_numbers_instyle = null_numbers.style.background_gradient(axis=0, gmap=null_numbers['Nan_number'], cmap='Blues')

In [101]:
nulls_numbers_instyle

,Column,Nan_number
7,1MReturn,12
8,3MReturn,8
9,6MReturn,11
10,12MReturn,341


In [102]:
dfi.export(nulls_numbers_instyle,r"Assets\nulls_in_returns.png")

c:\Users\Krzysztof\AppData\Local\Programs\Python\Python310\lib\site-packages\dataframe_image\_pandas_accessor.py:69: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



There is not many nans in returns. It can be dropped

In [221]:
convictions_test.dropna(subset=['1MReturn','3MReturn','6MReturn', '12MReturn'], how='any', inplace = True)

In [222]:
convictions_test.isnull().sum(axis = 0)

DATE           0
ID           205
STOCK          0
SECTOR       205
TYPE         205
SCORE          0
TIME         205
1MReturn       0
3MReturn       0
6MReturn       0
12MReturn      0
dtype: int64

In [220]:
convictions_test.to_csv(r'Assets\data_to_forecast.csv')